In [15]:
import os
import json
import boto3
import random
from time import time
from datetime import datetime
from typing import List, Dict, Any
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever


In [16]:

load_dotenv()

class Config:
    AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
    AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
    KB_ID = os.getenv("KB_ID")
    MODEL_ARN = os.getenv("MODEL_ARN")
    DYNAMODB_TABLE_NAME = os.getenv("DYNAMODB_TABLE_NAME")
    AWS_REGION= os.getenv("AWS_REGION", "us-east-1")

In [17]:
# Initialize AWS Bedrock Runtime Client
client = boto3.client("bedrock-agent-runtime", region_name=Config.AWS_REGION)

In [18]:
def rewrite_question(user_input: str, history_context: str, llm=None) -> str:
    """
    (Optional) Rewrite the user's input as a standalone question using LLM.
    """
    if not llm:
        return user_input  # skip if no LLM available

    prompt = f"""Rewrite the user's question to be self-contained based on the chat history.

    Chat history:
    {history_context}

    User question: {user_input}

    Standalone question:"""

    rewritten = llm.invoke(prompt.strip())
    return rewritten.content if hasattr(rewritten, "content") else rewritten

In [19]:
def get_dynamic_prompt(user_input: str, history: List) -> PromptTemplate:
    sensitive_keywords = ["complaint", "harassment", "grievance", "termination"]
    policy_keywords = ["policy", "rule", "guideline"]
    benefit_keywords = ["benefit", "pto", "leave", "insurance"]
    
    if any(kw in user_input.lower() for kw in sensitive_keywords):
        instructions = "This is a sensitive topic. Be professional and direct the user to official HR channels if appropriate."
    elif any(kw in user_input.lower() for kw in policy_keywords):
        instructions = "Provide exact policy details with reference to the policy document when possible."
    elif any(kw in user_input.lower() for kw in benefit_keywords):
        instructions = "Include eligibility requirements and any limitations for benefits mentioned."
    else:
        instructions = "Respond helpfully and professionally."
    
    template = f"""You are an HR assistant for a company. Use the following context to answer the question at the end.
If you don't know the answer, say you don't know. Be concise but helpful.

Context:
{{context}}

Conversation history:
{{chat_history}}

Question: {{input}}

Considerations:
1. {instructions}
2. Format lists and important details clearly
3. Provide sources when available

Answer:"""
    
    return PromptTemplate.from_template(template)


In [20]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model="anthropic.claude-3-5-sonnet-20240620-v1:0",
    region_name=Config.AWS_REGION,
    beta_use_converse_api=True,
    streaming=True
)

In [ ]:
def summarize_chat_history(chat_history: list) -> str:
    text = ""
    for entry in chat_history:
        if not entry.get("summarized"):
            text += f"User: {entry['user_message']}\nAssistant: {entry['assistant_response']}\n"

    prompt = f"""Summarize the following HR chat in 1-2 concise sentences:

{text}

Summary:"""

    return llm.invoke(prompt).content.strip()


In [7]:
from boto3.dynamodb.conditions import Key,Attr
dynamodb = boto3.resource('dynamodb',region_name=Config.AWS_REGION,)
TABLE_NAME = Config.DYNAMODB_TABLE_NAME
table = dynamodb.Table(TABLE_NAME)

class ChatHistoryManager:
    def __init__(self, user_id: str = "default", session_id: str = "default_session"):
        self.user_id = user_id
        self.session_id = session_id

    def load_history(self) -> List[Dict[str, Any]]:
        today = datetime.now().date()
        start_time = datetime.combine(today, datetime.min.time()).isoformat()
        end_time = datetime.combine(today, datetime.max.time()).isoformat()

        try:
            response = table.query(
                KeyConditionExpression=Key('user_id').eq(self.user_id) & Key('timestamp').between(start_time, end_time),
                FilterExpression=Attr('session_id').eq(self.session_id)
            )
            return response.get('Items', [])
        except Exception as e:
            print("Error loading from DynamoDB:", e)
            return []

    def save_history(self, history: List[Dict[str, Any]]):
        for entry in history:
            item = {
                'user_id': self.user_id,
                'timestamp': entry.get("timestamp", datetime.now().isoformat()),  # sort key
                'session_id': entry.get("session_id", self.session_id),
                'user_message': entry.get("user_message", ""),
                'assistant_response': entry.get("assistant_response", ""),
                'summarized': entry.get("summarized", False)
            }

            if "summary" in entry:
                item["summary"] = entry["summary"]
            if "summary_of" in entry:
                item["summary_of"] = entry["summary_of"]

            try:
                table.put_item(Item=item)
            except Exception as e:
                print("Error writing to DynamoDB:", e)

    def summarize_if_needed(self, history: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        unsummarized_blocks = [
            entry for entry in history
            if not entry.get("summarized", False)
            and entry.get("user_message") and entry.get("assistant_response")
        ]

        if len(unsummarized_blocks) < 5:
            return history

        history_text = "\n".join(
            f"User: {entry['user_message']}\nAssistant: {entry['assistant_response']}"
            for entry in unsummarized_blocks[:10]
        )

        summary_prompt = f"""
        Summarize the following 10 interactions into one concise but informative summary:
        {history_text}
        """

        summary = llm.invoke(summary_prompt.strip())
        if isinstance(summary, AIMessage):
            summary = summary.content

        summary_entry = {
            "role": "system",
            "summary": summary,
            "timestamp": datetime.now().isoformat(),
            "session_id": self.session_id,
            "summarized": True,
            "summary_of": [entry["timestamp"] for entry in unsummarized_blocks[:10]]
        }

        history = [entry for entry in history if entry not in unsummarized_blocks[:10]]
        history.insert(0, summary_entry)
        return history

    def append_chat_pair(self, history: List[Dict[str, Any]], user_msg: str, assistant_msg: str) -> List[Dict[str, Any]]:
        new_entry = {
            "user_message": user_msg,
            "assistant_response": assistant_msg,
            "timestamp": datetime.now().isoformat(),
            "session_id": self.session_id,
            "summarized": False
        }

        history.append(new_entry)
        self.save_history([new_entry])  # Save only the new one
        return self.summarize_if_needed(history)

In [8]:
def chat_with_bedrock(user_input: str, user_id: str = "default", session_id: str = "default_session") -> str:
    history_manager = ChatHistoryManager(user_id, session_id)
    chat_history = history_manager.load_history()

    # --- Format recent history for context ---
    formatted_history = []
    for entry in chat_history:
        if entry.get("summarized", False):
            formatted_history.append(f"System Summary: {entry['summary']}")
        else:
            formatted_history.append(f"User: {entry['user_message']}\nAssistant: {entry['assistant_response']}")

    history_context = "\n".join(formatted_history[-10:])  # last 10 messages only

    # --- Build dynamic system prompt ---
    system_prompt = get_dynamic_prompt(user_input, chat_history).format(
        context="Knowledge from internal HR documents.",
        chat_history=history_context,
        input=user_input
    )

    # --- (Optional) Rewrite question using context ---
    rewritten_question = user_input  # rewrite_question(user_input, history_context)

    # --- Call Bedrock retrieve_and_generate ---
    try:
        response = client.retrieve_and_generate(
            input={'text': rewritten_question},
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': Config.KB_ID,
                    'modelArn': Config.MODEL_ARN
                }
            }
        )

        answer = response.get("output", {}).get("text", "Sorry, I don't have an answer for that.")

        # Save chat entry
        chat_history = history_manager.append_chat_pair(chat_history, user_input, answer)

        # Summarize every 10 entries
        if len([h for h in chat_history if not h.get("summarized")]) >= 10:
            summary_text = summarize_chat_history(chat_history)
            summary_entry = {
                "id": str(uuid.uuid4()),
                "timestamp": datetime.datetime.utcnow().isoformat(),
                "summary": summary_text,
                "summarized": True
            }
            chat_history.append(summary_entry)

        history_manager.save_history(chat_history)

        return answer

    except Exception as e:
        print(f"[Error invoking Bedrock] {e}")
        return "Sorry, I couldn't retrieve the information right now."


In [23]:
response = chat_with_bedrock("Can I carry over my unused vacation days?", user_id="john", session_id="abc123")
print(response)

Yes, you can carry over unused vacation days, but there are some limitations. If you work full-time on the India payroll, you are allowed to carry over up to 10 days of Earned leaves to the following year. However, there is a maximum limit of 30 leave balances that can be accumulated.

It's important to note that the company prefers staff to take their full holiday entitlement in the leave year to which it relates. The holiday year runs from January 1 to December 31.


In [11]:
chat_with_bedrock("what do you know about the CEO of AyataCommerce?",user_id="123",session_id="1234")

'The CEO and Founder of AyataCommerce is Shine Mathew. Unfortunately, the search results do not provide much detailed information about Shine beyond mentioning his role and name.'

In [13]:
chat_with_bedrock("what do you know about him?",user_id="123",session_id="1234")

'Sorry, I am unable to assist you with this request.'

In [20]:
chat("What are the core values of the company?","123","1234")

"The core values of AyataCommerce are:\n\n1. **Empathy**: Emphasizing understanding and valuing diverse perspectives within the global team.\n2. **Trust**: Building a transparent and respectful environment that fosters mutual trust.\n3. **Adaptability**: Encouraging flexibility and openness to innovative ideas and changes.\n\nThese values guide the company's working practices and interactions with clients and each other."

In [ ]:
chat("Which among the values reflects about the flexibility?","123")

Error in RAG chain: Object of type AIMessage is not JSON serializable


'My knowledge base seems to be unavailable at the moment. Please try again later.'

In [61]:
chat("Is it ok to take a half day leave if i am sick suddenly?","123","1234")

"Yes, you can take a half-day leave if you are suddenly sick. Here are the key points to consider:\n\n### Eligibility Requirements:\n1. **Notification**: You must inform your manager as soon as possible on the first day of your absence.\n2. **Duration of Absence**: If you are going to be away from work for more than 48 hours, you must notify your manager and may need to provide a medical certificate if the absence extends beyond two days.\n\n### Limitations:\n- **Sick Pay**: Ensure you understand the rules regarding company sick pay. If you have any uncertainties, it's advisable to ask payroll for clarification.\n- **Documentation**: If your illness continues beyond the term of your medical certificate, further documentation may be required to confirm your sickness.\n\n### Additional Notes:\n- You should record your leave accurately in Zoho.\n- If you are sick during a holiday, you must notify your manager promptly to potentially treat it as sick leave rather than annual leave.\n\nFor 